In [1]:
# Cell 1: Initial setup, connecting to Google Drive, installing libraries, and checking GPU availability.
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Instalar y actualizar las librerías
print("\nInstalando y actualizando librerías...")
!pip install --upgrade -q mne pytorch-lightning timm
print("✅ Librerías listas.")

# Step 3: Prueba explícita de control de la GPU
import torch
print("\n--- INICIANDO PRUEBA DE CONTROL DE GPU ---")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
    try:
        tensor_grande = torch.randn(1024, 1024, 512, device=device) # Asignar 2GB
        memoria_asignada = torch.cuda.memory_allocated(0) / 1024**3
        print(f"✅ ¡Éxito! Memoria asignada activamente: {memoria_asignada:.2f} GB")
        del tensor_grande
        torch.cuda.empty_cache()
        print("✅ Memoria liberada correctamente.")
        print("--- PRUEBA DE CONTROL DE GPU COMPLETADA EXITOSAMENTE ---")
    except Exception as e:
        print(f"❌ ¡ERROR DURANTE LA PRUEBA! No se pudo asignar memoria a la GPU: {e}")
else:
    print("❌ ¡ERROR! No se detectó ninguna GPU en este entorno de ejecución.")

Mounted at /content/drive

Instalando y actualizando librerías...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 59.3 MB/s eta 0:00:00
✅ Librerías listas.

--- INICIANDO PRUEBA DE CONTROL DE GPU ---
✅ GPU detectada: NVIDIA A100-SXM4-40GB
✅ ¡Éxito! Memoria asignada activamente: 2.00 GB
✅ Memoria liberada correctamente.
--- PRUEBA DE CONTROL DE GPU COMPLETADA EXITOSAMENTE ---


In [6]:
# ==============================================================================
# 1. SETUP AND DEPENDENCY INSTALLATION
# ==============================================================================
print("Ensuring PyTorch Lightning and other libraries are installed...")
# Install the necessary libraries with pinned versions to avoid conflicts
!pip install --upgrade -q pytorch-lightning timm "pandas==2.2.2" "pyarrow==19.0.0"
print("✅ Installation check complete.")

# ==============================================================================
# 2. IMPORTS AND INITIAL CONFIGURATION
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
import numpy as np
import pandas as pd
from pathlib import Path
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Set matrix multiplication precision for A100/H100 GPUs for better performance
torch.set_float32_matmul_precision('medium')
print("✅ Libraries imported and configuration set.")

# ==============================================================================
# 3. MODEL ARCHITECTURE DEFINITION
# ==============================================================================
def get_model(model_name='convnext_tiny', num_classes=5, pretrained=True):
    """
    Creates a model adapted for sleep stage classification.
    """
    if model_name == 'convnext_tiny':
        model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Tiny model created.")
    else:
        raise ValueError(f"Model '{model_name}' not supported for this script.")

    return model

print("✅ `get_model` function defined.")

# ==============================================================================
# 4. PYTORCH LIGHTNING MODULE
# ==============================================================================
class SleepStageClassifierLightning(pl.LightningModule):
    def __init__(self, model_name, learning_rate=1e-5, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_model(model_name=self.hparams.model_name, num_classes=5, pretrained=True)
        self.train_accuracy = MulticlassAccuracy(num_classes=5)
        self.val_accuracy = MulticlassAccuracy(num_classes=5)
        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        # --- MODIFICATION: Use the functional version of the loss function ---
        # This is more flexible and avoids state_dict issues when resuming.
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        # --- MODIFICATION: Pass weights directly to the loss function ---
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.train_accuracy(y_pred_logits, y_true), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        # --- MODIFICATION: Pass weights directly to the loss function ---
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_accuracy(y_pred_logits, y_true), on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3),
            'monitor': 'val_loss',
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

print("✅ `SleepStageClassifierLightning` module defined.")

# ==============================================================================
# 5. CUSTOM DATASET DEFINITION
# ==============================================================================
class CombinedDataset(Dataset):
    def __init__(self, file_paths_chunk):
        print(f"Initializing dataset with {len(file_paths_chunk)} files...")
        self.file_paths = file_paths_chunk
        self.epochs_per_file = []
        self._cache = {}
        for f_path in self.file_paths:
            try:
                df_labels = pd.read_parquet(f_path, columns=['label'])
                num_valid = df_labels['label'].isin([0, 1, 2, 3, 4]).sum()
                self.epochs_per_file.append(num_valid)
            except Exception as e:
                print(f"Warning: Could not read or process {f_path.name}. Skipping. Error: {e}")
                self.epochs_per_file.append(0)
        self.cumulative_epochs = np.cumsum(self.epochs_per_file)
        self.total_epochs = self.cumulative_epochs[-1] if len(self.cumulative_epochs) > 0 else 0
        print(f"✅ Dataset initialized. Total valid epochs: {self.total_epochs}")

    def __len__(self):
        return self.total_epochs

    def __getitem__(self, idx):
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)
        file_path = self.file_paths[file_idx]
        if file_path not in self._cache:
            try:
                df = pd.read_parquet(file_path)
                self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)
            except Exception as e:
                raise IOError(f"Error reading file {file_path.name} in __getitem__: {e}")
        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)
        mean, std = spectrogram_flat.mean(), spectrogram_flat.std()
        spectrogram_normalized = (spectrogram_flat - mean) / (std + 1e-6)
        spectrogram_2d = spectrogram_normalized.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

print("✅ `CombinedDataset` class defined.")

# ==============================================================================
# 6. TRAINING EXECUTION
# ==============================================================================
print("\n--- Starting ConvNeXT Tiny Experiment (1000 Files, Resumable) ---")

# --- General Parameters ---
MODEL_TO_TRAIN = 'convnext_tiny'
EPOCHS = 40
BATCH_SIZE = 256
# --- MODIFICATION: Reduced number of workers to prevent memory errors ---
NUM_WORKERS = 2
CLASS_WEIGHTS = [0.7, 3.5, 0.5, 1.5, 1.2]
LEARNING_RATE = 5e-5

# --- Paths and File Identification (using Google Drive) ---
shhs1_processed_dir_base = Path('/content/drive/MyDrive/shhs1_processed')
shhs2_processed_dir_base = Path('/content/drive/MyDrive/shhs2_processed')

# Using 1000 files for a fair comparison
shhs1_files = list(shhs1_processed_dir_base.glob('*.parquet'))[:500]
shhs2_files = list(shhs2_processed_dir_base.glob('*.parquet'))[:500]
specific_shhs_file_paths = shhs1_files + shhs2_files

# --- Main Experiment Logic ---
if not specific_shhs_file_paths:
     print("\nERROR: No valid .parquet files were found. Aborting experiment.")
else:
    print(f"\nFound {len(specific_shhs_file_paths)} specific files for training.")

    full_dataset = CombinedDataset(specific_shhs_file_paths)

    if len(full_dataset) > 1:
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

        print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)

        model = SleepStageClassifierLightning(
            model_name=MODEL_TO_TRAIN,
            learning_rate=LEARNING_RATE,
            class_weights=CLASS_WEIGHTS
        )

        experiment_name = f"{MODEL_TO_TRAIN}_1000_files_resumable"
        checkpoint_dir = Path('/content/drive/MyDrive/final_model_checkpoint/')
        csv_logger = CSVLogger("/content/drive/MyDrive/sleep_logs/", name=experiment_name)

        # --- MODIFIED CHECKPOINT CALLBACK ---
        # save_last=True creates the 'last.ckpt' file for resuming.
        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=checkpoint_dir,
            filename=f"sleep-stage-model-{experiment_name}-{{epoch:02d}}-{{val_loss:.4f}}",
            save_top_k=1,
            mode='min',
            save_last=True # <-- IMPORTANT FOR RESUMING
        )

        early_stop_callback = EarlyStopping(
           monitor='val_loss',
           patience=7,
           verbose=True,
           mode='min'
        )

        trainer = pl.Trainer(
            max_epochs=EPOCHS,
            accelerator="gpu",
            devices=1,
            logger=csv_logger,
            callbacks=[checkpoint_callback, early_stop_callback],
            precision="bf16-mixed",
            gradient_clip_val=1.0
        )

        # --- RESUME LOGIC ---
        # Check if a 'last.ckpt' file exists to resume from.
        resume_checkpoint_path = checkpoint_dir / "last.ckpt"
        if resume_checkpoint_path.exists():
            print(f"🚀 Resuming training from checkpoint: {resume_checkpoint_path}")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader, ckpt_path=resume_checkpoint_path)
        else:
            print(f"🚀 Starting new training run for {MODEL_TO_TRAIN.upper()}...")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        print(f"✅ Training complete for {MODEL_TO_TRAIN.upper()}!")
        print(f"Best model for this run saved at: {checkpoint_callback.best_model_path}")

    else:
        print("Dataset is too small to split. Aborting experiment.")


Ensuring PyTorch Lightning and other libraries are installed...
✅ Installation check complete.
✅ Libraries imported and configuration set.
✅ `get_model` function defined.
✅ `SleepStageClassifierLightning` module defined.
✅ `CombinedDataset` class defined.

--- Starting ConvNeXT Tiny Experiment (1000 Files, Resumable) ---

Found 1000 specific files for training.
Initializing dataset with 1000 files...
✅ Dataset initialized. Total valid epochs: 1093021
Dataset split: 874416 training samples, 218605 validation samples.


INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ConvNeXt           | 27.9 M | train
1 | train_accuracy | MulticlassAccuracy | 0      | train
2 | val_accuracy   | MulticlassAccuracy | 0      | train
--------------------------------------------------------------
27.9 M    Trainable params
0         Non-trainable params
27.9 M    Total params
111.469   Total estimated model params size (MB)
251       Modules in train mode
0         Modules 

✅ ConvNeXT Tiny model created.
🚀 Starting new training run for CONVNEXT_TINY...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1054, in _run_stage
    self._run_sanity_check()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1083, in _run_sanity_check
    val_loop.run()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/utilities.py", line 179, in _decorator
    return loop_run(self, *args, **kwargs)
         

TypeError: object of type 'NoneType' has no len()

In [ ]:
# ==============================================================================
# 1. SETUP AND DEPENDENCY INSTALLATION
# ==============================================================================
print("Ensuring PyTorch Lightning and other libraries are installed...")
# Install the necessary libraries with pinned versions to avoid conflicts
!pip install --upgrade -q pytorch-lightning timm "pandas==2.2.2" "pyarrow==19.0.0"
print("✅ Installation check complete.")

# ==============================================================================
# 2. IMPORTS AND INITIAL CONFIGURATION
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
import numpy as np
import pandas as pd
from pathlib import Path
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Set matrix multiplication precision for A100/H100 GPUs for better performance
torch.set_float32_matmul_precision('medium')
print("✅ Libraries imported and configuration set.")

# ==============================================================================
# 3. MODEL ARCHITECTURE DEFINITION
# ==============================================================================
def get_model(model_name='convnext_tiny', num_classes=5, pretrained=True):
    """
    Creates a model adapted for sleep stage classification.
    """
    if model_name == 'convnext_tiny':
        model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Tiny model created.")
    else:
        raise ValueError(f"Model '{model_name}' not supported for this script.")

    return model

print("✅ `get_model` function defined.")

# ==============================================================================
# 4. PYTORCH LIGHTNING MODULE
# ==============================================================================
class SleepStageClassifierLightning(pl.LightningModule):
    def __init__(self, model_name, learning_rate=1e-5, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_model(model_name=self.hparams.model_name, num_classes=5, pretrained=True)
        self.train_accuracy = MulticlassAccuracy(num_classes=5)
        self.val_accuracy = MulticlassAccuracy(num_classes=5)
        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        # Use the functional version of the loss function for robustness
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.train_accuracy(y_pred_logits, y_true), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_accuracy(y_pred_logits, y_true), on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3),
            'monitor': 'val_loss',
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

print("✅ `SleepStageClassifierLightning` module defined.")

# ==============================================================================
# 5. CUSTOM DATASET DEFINITION
# ==============================================================================
class CombinedDataset(Dataset):
    def __init__(self, file_paths_chunk):
        print(f"Initializing dataset with {len(file_paths_chunk)} files...")
        self.file_paths = file_paths_chunk
        self.epochs_per_file = []
        self._cache = {}
        for f_path in self.file_paths:
            try:
                df_labels = pd.read_parquet(f_path, columns=['label'])
                num_valid = df_labels['label'].isin([0, 1, 2, 3, 4]).sum()
                self.epochs_per_file.append(num_valid)
            except Exception as e:
                print(f"Warning: Could not read or process {f_path.name}. Skipping. Error: {e}")
                self.epochs_per_file.append(0)
        self.cumulative_epochs = np.cumsum(self.epochs_per_file)
        self.total_epochs = self.cumulative_epochs[-1] if len(self.cumulative_epochs) > 0 else 0
        print(f"✅ Dataset initialized. Total valid epochs: {self.total_epochs}")

    def __len__(self):
        return self.total_epochs

    def __getitem__(self, idx):
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)
        file_path = self.file_paths[file_idx]
        if file_path not in self._cache:
            try:
                df = pd.read_parquet(file_path)
                self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)
            except Exception as e:
                raise IOError(f"Error reading file {file_path.name} in __getitem__: {e}")
        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)
        mean, std = spectrogram_flat.mean(), spectrogram_flat.std()
        spectrogram_normalized = (spectrogram_flat - mean) / (std + 1e-6)
        spectrogram_2d = spectrogram_normalized.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

print("✅ `CombinedDataset` class defined.")

# ==============================================================================
# 6. TRAINING EXECUTION
# ==============================================================================
print("\n--- Starting ConvNeXT Tiny Experiment (1000 Files, Resumable) ---")

# --- General Parameters ---
MODEL_TO_TRAIN = 'convnext_tiny'
EPOCHS = 40
BATCH_SIZE = 256
NUM_WORKERS = 2 # Reduced to prevent memory errors
CLASS_WEIGHTS = [0.7, 3.5, 0.5, 1.5, 1.2]
# --- MODIFICATION: Reduced learning rate for stability on a larger dataset ---
# 1e-4: This learning rate was tested during your learning rate tuning experiment. It caused the model to overfit quickly and resulted in a higher validation loss (0.6697) than the other options.
# 5e-5: While this was the winning rate for the 100-file dataset, it was too high for the larger 1000-file dataset. It caused the training to become unstable and resulted in very poor performance, with a final validation loss of 1.28.
LEARNING_RATE = 5e-6

# --- Paths and File Identification (using Google Drive) ---
shhs1_processed_dir_base = Path('/content/drive/MyDrive/shhs1_processed')
shhs2_processed_dir_base = Path('/content/drive/MyDrive/shhs2_processed')

# Using 1000 files for a fair comparison
shhs1_files = list(shhs1_processed_dir_base.glob('*.parquet'))[:500]
shhs2_files = list(shhs2_processed_dir_base.glob('*.parquet'))[:500]
specific_shhs_file_paths = shhs1_files + shhs2_files

# --- Main Experiment Logic ---
if not specific_shhs_file_paths:
     print("\nERROR: No valid .parquet files were found. Aborting experiment.")
else:
    print(f"\nFound {len(specific_shhs_file_paths)} specific files for training.")

    full_dataset = CombinedDataset(specific_shhs_file_paths)

    if len(full_dataset) > 1:
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

        print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)

        model = SleepStageClassifierLightning(
            model_name=MODEL_TO_TRAIN,
            learning_rate=LEARNING_RATE,
            class_weights=CLASS_WEIGHTS
        )

        experiment_name = f"{MODEL_TO_TRAIN}_1000_files_resumable_lr_5e-6"
        checkpoint_dir = Path('/content/drive/MyDrive/final_model_checkpoint/')
        csv_logger = CSVLogger("/content/drive/MyDrive/sleep_logs/", name=experiment_name)

        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=checkpoint_dir,
            filename=f"sleep-stage-model-{experiment_name}-{{epoch:02d}}-{{val_loss:.4f}}",
            save_top_k=1,
            mode='min',
            save_last=True
        )

        early_stop_callback = EarlyStopping(
           monitor='val_loss',
           patience=7,
           verbose=True,
           mode='min'
        )

        trainer = pl.Trainer(
            max_epochs=EPOCHS,
            accelerator="gpu",
            devices=1,
            logger=csv_logger,
            callbacks=[checkpoint_callback, early_stop_callback],
            precision="bf16-mixed",
            gradient_clip_val=1.0
        )

        # --- RESUME LOGIC ---
        resume_checkpoint_path = checkpoint_dir / "last.ckpt"
        if resume_checkpoint_path.exists():
            print(f"🚀 Resuming training from checkpoint: {resume_checkpoint_path}")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader, ckpt_path=resume_checkpoint_path)
        else:
            print(f"🚀 Starting new training run for {MODEL_TO_TRAIN.upper()}...")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        print(f"✅ Training complete for {MODEL_TO_TRAIN.upper()}!")
        print(f"Best model for this run saved at: {checkpoint_callback.best_model_path}")

    else:
        print("Dataset is too small to split. Aborting experiment.")

In [ ]:
# ==============================================================================
# 1. SETUP AND DEPENDENCY INSTALLATION
# ==============================================================================
print("Ensuring PyTorch Lightning and other libraries are installed...")
# Install the necessary libraries with pinned versions to avoid conflicts
!pip install --upgrade -q pytorch-lightning timm "pandas==2.2.2" "pyarrow==19.0.0"
print("✅ Installation check complete.")

# ==============================================================================
# 2. IMPORTS AND INITIAL CONFIGURATION
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
import numpy as np
import pandas as pd
from pathlib import Path
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Set matrix multiplication precision for A100/H100 GPUs for better performance
torch.set_float32_matmul_precision('medium')
print("✅ Libraries imported and configuration set.")

# ==============================================================================
# 3. MODEL ARCHITECTURE DEFINITION
# ==============================================================================
def get_model(model_name='convnext_tiny', num_classes=5, pretrained=True):
    """
    Creates a model adapted for sleep stage classification.
    """
    if model_name == 'convnext_tiny':
        model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Tiny model created.")

    elif model_name == 'convnext_base':
        model = timm.create_model('convnextv2_base.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Base model created.")

    else:
        raise ValueError(f"Model '{model_name}' not supported for this script.")

    return model

print("✅ `get_model` function defined.")

# ==============================================================================
# 4. PYTORCH LIGHTNING MODULE
# ==============================================================================
class SleepStageClassifierLightning(pl.LightningModule):
    def __init__(self, model_name, learning_rate=1e-5, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_model(model_name=self.hparams.model_name, num_classes=5, pretrained=True)
        self.train_accuracy = MulticlassAccuracy(num_classes=5)
        self.val_accuracy = MulticlassAccuracy(num_classes=5)
        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.train_accuracy(y_pred_logits, y_true), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_accuracy(y_pred_logits, y_true), on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3),
            'monitor': 'val_loss',
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

print("✅ `SleepStageClassifierLightning` module defined.")

# ==============================================================================
# 5. CUSTOM DATASET DEFINITION
# ==============================================================================
class CombinedDataset(Dataset):
    def __init__(self, file_paths_chunk):
        print(f"Initializing dataset with {len(file_paths_chunk)} files...")
        self.file_paths = file_paths_chunk
        self.epochs_per_file = []
        self._cache = {}
        for f_path in self.file_paths:
            try:
                df_labels = pd.read_parquet(f_path, columns=['label'])
                num_valid = df_labels['label'].isin([0, 1, 2, 3, 4]).sum()
                self.epochs_per_file.append(num_valid)
            except Exception as e:
                print(f"Warning: Could not read or process {f_path.name}. Skipping. Error: {e}")
                self.epochs_per_file.append(0)
        self.cumulative_epochs = np.cumsum(self.epochs_per_file)
        self.total_epochs = self.cumulative_epochs[-1] if len(self.cumulative_epochs) > 0 else 0
        print(f"✅ Dataset initialized. Total valid epochs: {self.total_epochs}")

    def __len__(self):
        return self.total_epochs

    def __getitem__(self, idx):
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)
        file_path = self.file_paths[file_idx]
        if file_path not in self._cache:
            try:
                df = pd.read_parquet(file_path)
                self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)
            except Exception as e:
                raise IOError(f"Error reading file {file_path.name} in __getitem__: {e}")
        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)
        mean, std = spectrogram_flat.mean(), spectrogram_flat.std()
        spectrogram_normalized = (spectrogram_flat - mean) / (std + 1e-6)
        spectrogram_2d = spectrogram_normalized.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

print("✅ `CombinedDataset` class defined.")

# ==============================================================================
# 6. TRAINING EXECUTION
# ==============================================================================
print("\n--- Starting ConvNeXT Base Experiment (1000 Files, Resumable) ---")

# --- General Parameters ---
MODEL_TO_TRAIN = 'convnext_base'
EPOCHS = 40
BATCH_SIZE = 256
# --- MODIFICATION: Reduced number of workers to prevent memory errors ---
NUM_WORKERS = 0
CLASS_WEIGHTS = [0.7, 3.5, 0.5, 1.5, 1.2]
LEARNING_RATE = 5e-6

# --- Paths and File Identification (using Google Drive) ---
shhs1_processed_dir_base = Path('/content/drive/MyDrive/shhs1_processed')
shhs2_processed_dir_base = Path('/content/drive/MyDrive/shhs2_processed')

# Using 1000 files for a fair comparison
shhs1_files = list(shhs1_processed_dir_base.glob('*.parquet'))[:500]
shhs2_files = list(shhs2_processed_dir_base.glob('*.parquet'))[:500]
specific_shhs_file_paths = shhs1_files + shhs2_files

# --- Main Experiment Logic ---
if not specific_shhs_file_paths:
     print("\nERROR: No valid .parquet files were found. Aborting experiment.")
else:
    print(f"\nFound {len(specific_shhs_file_paths)} specific files for training.")

    full_dataset = CombinedDataset(specific_shhs_file_paths)

    if len(full_dataset) > 1:
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

        print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers= (NUM_WORKERS > 0) )
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers= (NUM_WORKERS > 0) )

        model = SleepStageClassifierLightning(
            model_name=MODEL_TO_TRAIN,
            learning_rate=LEARNING_RATE,
            class_weights=CLASS_WEIGHTS
        )

        experiment_name = f"{MODEL_TO_TRAIN}_1000_files_resumable_lr_5e-6"
        checkpoint_dir = Path('/content/drive/MyDrive/final_model_checkpoint/')
        csv_logger = CSVLogger("/content/drive/MyDrive/sleep_logs/", name=experiment_name)

        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=checkpoint_dir,
            filename=f"sleep-stage-model-{experiment_name}-{{epoch:02d}}-{{val_loss:.4f}}",
            save_top_k=1,
            mode='min',
            save_last=True
        )

        early_stop_callback = EarlyStopping(
           monitor='val_loss',
           patience=7,
           verbose=True,
           mode='min'
        )

        trainer = pl.Trainer(
            max_epochs=EPOCHS,
            accelerator="gpu",
            devices=1,
            logger=csv_logger,
            callbacks=[checkpoint_callback, early_stop_callback],
            precision="bf16-mixed",
            gradient_clip_val=1.0
        )

        # --- RESUME LOGIC ---
        resume_checkpoint_path = checkpoint_dir / "last.ckpt"
        if resume_checkpoint_path.exists():
            print(f"🚀 Resuming training from checkpoint: {resume_checkpoint_path}")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader, ckpt_path=resume_checkpoint_path)
        else:
            print(f"🚀 Starting new training run for {MODEL_TO_TRAIN.upper()}...")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        print(f"✅ Training complete for {MODEL_TO_TRAIN.upper()}!")
        print(f"Best model for this run saved at: {checkpoint_callback.best_model_path}")

    else:
        print("Dataset is too small to split. Aborting experiment.")

Ensuring PyTorch Lightning and other libraries are installed...
✅ Installation check complete.
✅ Libraries imported and configuration set.
✅ `get_model` function defined.
✅ `SleepStageClassifierLightning` module defined.
✅ `CombinedDataset` class defined.

--- Starting ConvNeXT Base Experiment (1000 Files, Resumable) ---

Found 1000 specific files for training.
Initializing dataset with 1000 files...
✅ Dataset initialized. Total valid epochs: 1093021
Dataset split: 874416 training samples, 218605 validation samples.


INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


✅ ConvNeXT Base model created.
🚀 Starting new training run for CONVNEXT_BASE...


INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ConvNeXt           | 87.7 M | train
1 | train_accuracy | MulticlassAccuracy | 0      | train
2 | val_accuracy   | MulticlassAccuracy | 0      | train
--------------------------------------------------------------
87.7 M    Trainable params
0         Non-trainable params
87.7 M    Total params
350.775   Total estimated model params size (MB)
467       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved. New best score: 0.817


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.109 >= min_delta = 0.0. New best score: 0.708


Validation: |          | 0/? [00:00<?, ?it/s]

In [2]:
# ==============================================================================
# 1. SETUP AND DEPENDENCY INSTALLATION
# ==============================================================================
print("Ensuring PyTorch Lightning and other libraries are installed...")
# Install the necessary libraries with pinned versions to avoid conflicts
!pip install --upgrade -q pytorch-lightning timm "pandas==2.2.2" "pyarrow==19.0.0"
print("✅ Installation check complete.")

# ==============================================================================
# 2. IMPORTS AND INITIAL CONFIGURATION
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
import numpy as np
import pandas as pd
from pathlib import Path
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Set matrix multiplication precision for A100/H100 GPUs for better performance
torch.set_float32_matmul_precision('medium')
print("✅ Libraries imported and configuration set.")

# ==============================================================================
# 3. MODEL ARCHITECTURE DEFINITION
# ==============================================================================
def get_model(model_name='convnext_tiny', num_classes=5, pretrained=True):
    """
    Creates a model adapted for sleep stage classification.
    """
    if model_name == 'convnext_tiny':
        model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Tiny model created.")

    elif model_name == 'convnext_base':
        model = timm.create_model('convnextv2_base.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Base model created.")

    else:
        raise ValueError(f"Model '{model_name}' not supported for this script.")

    return model

print("✅ `get_model` function defined.")

# ==============================================================================
# 4. PYTORCH LIGHTNING MODULE
# ==============================================================================
class SleepStageClassifierLightning(pl.LightningModule):
    def __init__(self, model_name, learning_rate=1e-5, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_model(model_name=self.hparams.model_name, num_classes=5, pretrained=True)
        self.train_accuracy = MulticlassAccuracy(num_classes=5)
        self.val_accuracy = MulticlassAccuracy(num_classes=5)
        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.train_accuracy(y_pred_logits, y_true), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device) if self.weights is not None else None)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_accuracy(y_pred_logits, y_true), on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3),
            'monitor': 'val_loss',
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

print("✅ `SleepStageClassifierLightning` module defined.")

# ==============================================================================
# 5. CUSTOM DATASET DEFINITION
# ==============================================================================
class CombinedDataset(Dataset):
    def __init__(self, file_paths_chunk):
        print(f"Initializing dataset with {len(file_paths_chunk)} files...")
        self.file_paths = file_paths_chunk
        self.epochs_per_file = []
        self._cache = {}
        for f_path in self.file_paths:
            try:
                df_labels = pd.read_parquet(f_path, columns=['label'])
                num_valid = df_labels['label'].isin([0, 1, 2, 3, 4]).sum()
                self.epochs_per_file.append(num_valid)
            except Exception as e:
                print(f"Warning: Could not read or process {f_path.name}. Skipping. Error: {e}")
                self.epochs_per_file.append(0)
        self.cumulative_epochs = np.cumsum(self.epochs_per_file)
        self.total_epochs = self.cumulative_epochs[-1] if len(self.cumulative_epochs) > 0 else 0
        print(f"✅ Dataset initialized. Total valid epochs: {self.total_epochs}")

    def __len__(self):
        return self.total_epochs

    def __getitem__(self, idx):
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)
        file_path = self.file_paths[file_idx]
        if file_path not in self._cache:
            try:
                df = pd.read_parquet(file_path)
                self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)
            except Exception as e:
                raise IOError(f"Error reading file {file_path.name} in __getitem__: {e}")
        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)
        mean, std = spectrogram_flat.mean(), spectrogram_flat.std()
        spectrogram_normalized = (spectrogram_flat - mean) / (std + 1e-6)
        spectrogram_2d = spectrogram_normalized.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

print("✅ `CombinedDataset` class defined.")

# ==============================================================================
# 6. TRAINING EXECUTION
# ==============================================================================
print("\n--- Starting ConvNeXT Base Experiment (1000 Files, Resumable) ---")

# --- General Parameters ---
MODEL_TO_TRAIN = 'convnext_base'
EPOCHS = 40
BATCH_SIZE = 256
NUM_WORKERS = 0
CLASS_WEIGHTS = [0.7, 3.5, 0.5, 1.5, 1.2]
# --- MODIFICATION: Using a balanced learning rate ---
LEARNING_RATE = 2e-5

# --- Paths and File Identification (using Google Drive) ---
shhs1_processed_dir_base = Path('/content/drive/MyDrive/shhs1_processed')
shhs2_processed_dir_base = Path('/content/drive/MyDrive/shhs2_processed')

# Using 1000 files for a fair comparison
shhs1_files = list(shhs1_processed_dir_base.glob('*.parquet'))[:500]
shhs2_files = list(shhs2_processed_dir_base.glob('*.parquet'))[:500]
specific_shhs_file_paths = shhs1_files + shhs2_files

# --- Main Experiment Logic ---
if not specific_shhs_file_paths:
     print("\nERROR: No valid .parquet files were found. Aborting experiment.")
else:
    print(f"\nFound {len(specific_shhs_file_paths)} specific files for training.")

    full_dataset = CombinedDataset(specific_shhs_file_paths)

    if len(full_dataset) > 1:
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

        print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers= (NUM_WORKERS > 0) )
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers= (NUM_WORKERS > 0) )

        model = SleepStageClassifierLightning(
            model_name=MODEL_TO_TRAIN,
            learning_rate=LEARNING_RATE,
            class_weights=CLASS_WEIGHTS
        )

        experiment_name = f"{MODEL_TO_TRAIN}_1000_files_resumable_lr_2e-5"
        checkpoint_dir = Path('/content/drive/MyDrive/final_model_checkpoint/')
        csv_logger = CSVLogger("/content/drive/MyDrive/sleep_logs/", name=experiment_name)

        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=checkpoint_dir,
            filename=f"sleep-stage-model-{experiment_name}-{{epoch:02d}}-{{val_loss:.4f}}",
            save_top_k=1,
            mode='min',
            save_last=True
        )

        early_stop_callback = EarlyStopping(
           monitor='val_loss',
           patience=7,
           verbose=True,
           mode='min'
        )

        trainer = pl.Trainer(
            max_epochs=EPOCHS,
            accelerator="gpu",
            devices=1,
            logger=csv_logger,
            callbacks=[checkpoint_callback, early_stop_callback],
            precision="bf16-mixed",
            gradient_clip_val=1.0
        )

        # --- RESUME LOGIC ---
        resume_checkpoint_path = checkpoint_dir / "last.ckpt"
        if resume_checkpoint_path.exists():
            print(f"🚀 Resuming training from checkpoint: {resume_checkpoint_path}")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader, ckpt_path=resume_checkpoint_path)
        else:
            print(f"🚀 Starting new training run for {MODEL_TO_TRAIN.upper()}...")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        print(f"✅ Training complete for {MODEL_TO_TRAIN.upper()}!")
        print(f"Best model for this run saved at: {checkpoint_callback.best_model_path}")

    else:
        print("Dataset is too small to split. Aborting experiment.")

Ensuring PyTorch Lightning and other libraries are installed...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 58.1 MB/s eta 0:00:00
✅ Installation check complete.
✅ Libraries imported and configuration set.
✅ `get_model` function defined.
✅ `SleepStageClassifierLightning` module defined.
✅ `CombinedDataset` class defined.

--- Starting ConvNeXT Base Experiment (1000 Files, Resumable) ---

Found 1000 specific files for training.
Initializing dataset with 1000 files...
✅ Dataset initialized. Total valid epochs: 59059
Dataset split: 47247 training samples, 11812 validation samples.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/355M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)


✅ ConvNeXT Base model created.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 7 records. Best score: 0.605. Signaling Trainer to stop.


✅ Training complete for CONVNEXT_BASE!
Best model for this run saved at: /content/drive/MyDrive/final_model_checkpoint/sleep-stage-model-convnext_base_1000_files_resumable_lr_2e-5-epoch=12-val_loss=0.6046.ckpt
